# Interruption du fou

Dans cet exercice, vous serez amené·e à calculer à la main le TF-IDF d’un terme dans un corpus avant de le comparer avec le résultat obtenu directement par la librairie *Scikit-Learn* pour mettre en évidence des variantes dans la méthodologie.

Vous utiliserez trois extraits de Nietzsche avec pour objet d’étude le mot *devoirs*. Après tout, c’est toujours amusant de citer un philosophe sans contexte !

> « Tous les hommes qui sentent qu’il leur faut les paroles et les intonations les plus violentes, les attitudes et les gestes les plus éloquents pour pouvoir agir, les politiciens révolutionnaires, les socialistes, les prédicateurs, avec ou sans christianisme, tous ceux qui veulent éviter les demi-succès : tous ceux-là parlent de *devoirs*, et toujours de devoirs qui ont un caractère absolu - autrement ils n’auraient point droit à leur pathos démesuré : ils le savent fort bien. » (*Le gai savoir*, I-5)

> « Il faut connaître non seulement la marche hardie, légère, délicate et rapide de ses propres pensées, mais avant tout la disposition aux grandes responsabilités, la hauteur et la profondeur du regard impérieux, le sentiment d’être séparé de la foule, des devoirs et des vertus de la foule, la protection et la défense bienveillante de ce qui est mal compris et calomnié, que ce soit Dieu ou le diable ; le penchant et l’habileté à la suprême justice, l’art du commandement, l’ampleur de la volonté, la lenteur du regard qui rarement admire, rarement se lève, et aime rarement… » (*Par-delà le bien et le mal*, VI-213)

> « Je vous le dis : il faut encore porter en soi un chaos, pour pouvoir mettre au monde une étoile dansante. Je vous le dis : vous portez encore un chaos en vous. » (*Ainsi parlait Zarathoustra*, I-5)

## Constituer un vocabulaire commun

Afin d’éviter de constater des écarts à la marge, consécutifs à l’utilisation de méthodes différentes pour compter les mots (pour *Scikit-Learn*, *l’habileté* vaut un seul mot), vous établirez un vocabulaire qui servira aux deux.

Commencez par charger le corpus en mémoire :

In [1]:
corpus = [
    "Tous les hommes qui sentent qu’il leur faut les paroles et les intonations les plus violentes, les attitudes et les gestes les plus éloquents pour pouvoir agir, les politiciens révolutionnaires, les socialistes, les prédicateurs, avec ou sans christianisme, tous ceux qui veulent éviter les demi-succès : tous ceux-là parlent de devoirs, et toujours de devoirs qui ont un caractère absolu - autrement ils n’auraient point droit à leur pathos démesuré : ils le savent fort bien.",
    "Il faut connaître non seulement la marche hardie, légère, délicate et rapide de ses propres pensées, mais avant tout la disposition aux grandes responsabilités, la hauteur et la profondeur du regard impérieux, le sentiment d’être séparé de la foule, des devoirs et des vertus de la foule, la protection et la défense bienveillante de ce qui est mal compris et calomnié, que ce soit Dieu ou le diable ; le penchant et l’habileté à la suprême justice, l’art du commandement, l’ampleur de la volonté, la lenteur du regard qui rarement admire, rarement se lève, et aime rarement…",
    "Je vous le dis : il faut encore porter en soi un chaos, pour pouvoir mettre au monde une étoile dansante. Je vous le dis : vous portez encore un chaos en vous."
]

Convertissez, dans une variable `doc_words`, chaque document en une liste de mots transformés en bas de casse, en filtrant les mots vides du français :

In [2]:
# your code here

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer('\w+')
fr_stopwords = stopwords.words('french')

doc_words = [
    [
        word.lower()
        for word in tokenizer.tokenize(doc)
        if word not in fr_stopwords
    ]
    for doc in corpus
]

Constituez à présent le vocabulaire de vos documents :

In [3]:
# your code here

vocabulary = set()

for doc in doc_words:
    for word in doc:
        vocabulary.add(word)

# at the end, a list
vocabulary = list(vocabulary)

## Une mesure TF-IDF à la main

L’objet de votre étude étant le mot *devoirs*, vous ne vous concentrerez que sur ce terme.

### La fréquence du terme

La formule qui permet de calculer $\text{TF}$ est la suivante :

$$
\text{TF}(t, d) = \frac{t}{d}
$$

Sachant que $d$ est le nombre de tokens du vocabulaire dans le document, quelle est la mesure TF de *devoirs* dans le corpus ?

In [4]:
# your code here

tfs = list()

for doc in doc_words:
    # number of words in the document
    n = len(doc)
    # occurrences of 'devoirs' set at 0
    c = 0
    # if 'devoirs' is found in the doc,
    # increase the count
    for word in doc:
        if word == "devoirs":
            c += 1
    tfs.append(c / n)

print(tfs)

[0.046511627906976744, 0.019230769230769232, 0.0]


### La fréquence inverse de document

La mesure IDF est régie par la formule ci-dessous :

$$
\text{IDF}(t) = \ln{\frac{N}{1 + \text{df}(t)}}
$$

Sachant que $N$ est le nombre total de documents dans le corpus et $d$ le nombre de fois où le terme *devoirs* apparaît, quel est son IDF ?

In [32]:
# your code here

from math import log as ln

N = len(doc_words)

d = sum([
    doc.count("devoirs")
    for doc in doc_words
])

idf = ln(N / (1 + d))

print(idf)

-0.2876820724517809


### Calcul du TF-IDF

Il reste maintenant à appliquer la formule pour chaque document du corpus :

$$
\text{TF-IDF}(t, d) = \text{TF} \cdot \text{IDF}
$$

In [35]:
# your code here

tfidfs = [
    tf * idf
    for tf in tfs
]

print(tfidfs)

[-0.5753641449035618, -0.2876820724517809, -0.0]


Des scores négatifs signifient que le mot est sur-représenté dans les documents, mais notre interprétation doit être soumise à 

## Application avec *Scikit-Learn*

Commencez par importer le transformateur `TfidfVectorizer` :

In [7]:
# your code here

from sklearn.feature_extraction.text import TfidfVectorizer

Avec le constructeur `TfidfVectorizer()`, créez une nouvelle instance de la classe, nommée `vectorizer`, en lui transmettant le vocabulaire défini plus haut :

In [8]:
# your code here

vectorizer = TfidfVectorizer(vocabulary=vocabulary)

Ajustez et transformez votre `vectorizer` dans une nouvelle variable `tfidf` grâce à la méthode `.fit_transform()` :

In [9]:
# your code here

tfidf = vectorizer.fit_transform(corpus)

Sans surprise, le résultat n’est pas facilement compréhensible en l’état :

In [10]:
print(tfidf)

  (0, 93)	0.13789542296930127
  (0, 90)	0.4136862689079038
  (0, 83)	0.13789542296930127
  (0, 80)	0.13789542296930127
  (0, 79)	0.08144325818367278
  (0, 78)	0.13789542296930127
  (0, 75)	0.13789542296930127
  (0, 73)	0.13789542296930127
  (0, 72)	0.13789542296930127
  (0, 71)	0.13789542296930127
  (0, 69)	0.13789542296930127
  (0, 67)	0.13789542296930127
  (0, 63)	0.13789542296930127
  (0, 61)	0.27579084593860254
  (0, 60)	0.13789542296930127
  (0, 59)	0.13789542296930127
  (0, 58)	0.13789542296930127
  (0, 57)	0.13789542296930127
  (0, 56)	0.13789542296930127
  (0, 53)	0.13789542296930127
  (0, 52)	0.13789542296930127
  (0, 50)	0.13789542296930127
  (0, 49)	0.27579084593860254
  (0, 39)	0.13789542296930127
  (0, 32)	0.10487302348517721
  :	:
  (1, 19)	0.12880347869939848
  (1, 16)	0.12880347869939848
  (1, 14)	0.12880347869939848
  (1, 13)	0.12880347869939848
  (1, 12)	0.12880347869939848
  (1, 9)	0.12880347869939848
  (1, 8)	0.12880347869939848
  (1, 5)	0.12880347869939848
  (1, 3)

Comme vous ne vous intéressez qu’au résultat pour le mot *devoirs*, recherchez son indice dans le vocabulaire. Utilisez pour cela l’attribut spécial `.vocabulary_` de votre `vectorizer` :

In [11]:
# your code here

vectorizer.vocabulary_['devoirs']

24

Il est désormais plus simple de rechercher la mesure TF-IDF dans chaque document en utilisant par exemple la méthode `.getrow()` :

In [12]:
print(tfidf.getrow(0))

  (0, 93)	0.13789542296930127
  (0, 90)	0.4136862689079038
  (0, 83)	0.13789542296930127
  (0, 80)	0.13789542296930127
  (0, 79)	0.08144325818367278
  (0, 78)	0.13789542296930127
  (0, 75)	0.13789542296930127
  (0, 73)	0.13789542296930127
  (0, 72)	0.13789542296930127
  (0, 71)	0.13789542296930127
  (0, 69)	0.13789542296930127
  (0, 67)	0.13789542296930127
  (0, 63)	0.13789542296930127
  (0, 61)	0.27579084593860254
  (0, 60)	0.13789542296930127
  (0, 59)	0.13789542296930127
  (0, 58)	0.13789542296930127
  (0, 57)	0.13789542296930127
  (0, 56)	0.13789542296930127
  (0, 53)	0.13789542296930127
  (0, 52)	0.13789542296930127
  (0, 50)	0.13789542296930127
  (0, 49)	0.27579084593860254
  (0, 39)	0.13789542296930127
  (0, 32)	0.10487302348517721
  (0, 31)	0.13789542296930127
  (0, 30)	0.08144325818367278
  (0, 29)	0.13789542296930127
  (0, 28)	0.13789542296930127
  (0, 24)	0.20974604697035443
  (0, 21)	0.13789542296930127
  (0, 18)	0.13789542296930127
  (0, 17)	0.13789542296930127
  (0, 15)	0

De manière plus concise, vous pouvez transposer la matrice afin d’interroger directement le numéro d’indice :

In [13]:
print(tfidf.T[62])

  (0, 1)	0.12880347869939848


Une autre manière plus parlante serait de passer par *Pandas* :

In [14]:
import pandas as pd

df = pd.DataFrame(
    tfidf.T.todense(),
    index=vectorizer.get_feature_names_out()
)

print(
    f"TF-IDF de 'devoirs' dans le document 0 : {df[0].devoirs}",
    f"TF-IDF de 'devoirs' dans le document 1 : {df[1].devoirs}",
    f"TF-IDF de 'devoirs' dans le document 2 : {df[2].devoirs}",
    sep="\n"
)

TF-IDF de 'devoirs' dans le document 0 : 0.20974604697035443
TF-IDF de 'devoirs' dans le document 1 : 0.09795836551893199
TF-IDF de 'devoirs' dans le document 2 : 0.0


Étonnant, non ? Quand vous aviez calculé à la main le TF-IDF du terme *devoirs* dans le corpus, vous aviez obtenu une mesure de 0 pour chaque document.

Dans la suite de l’exercice, vous allez comprendre que *Scikit-Learn* utilise une variante du calcul de la mesure avant de rajouter une étape de normalisation.

## Le fin mot de l’histoire

Les ajustements de *Scikit-Learn* se situent en fait à plusieurs niveaux :

- déjà, dans la formule TF-IDF, TF vaut pour la fréquence brute ;
- ensuite, afin d’éviter les divisions par zéro, il ajoute par défaut un document où figurerait tous les termes du vocabulaire ;
- enfin, il applique en sortie une normalisation euclidienne.

### Calcul de la fréquence brute

Dans le cadre de votre analyse sur le terme *devoirs*, il apparaît ainsi que son TF vaut pour les documents du corpus :

In [15]:
tfs = [
    doc.count("devoirs")
    for doc in corpus
]

print(tfs)

[2, 1, 0]


### Calcul de l’IDF lissé

Lors du paramétrage du transformateur, un paramètre `smooth_idf` est fixé par défaut à `True`, ce qui active une variante du calcul IDF :

$$
\text{IDF}(d, N) = \ln{\frac{1 + N}{1 + d}} + 1
$$

Pour le terme *devoirs*, la mesure IDF devient alors :

In [16]:
N = len(doc_words)

d = sum([
    doc.count("devoirs")
    for doc in doc_words
])

idf = ln( (1 + N) / (1 + d) ) + 1

print(idf)

1.0


**Remarque :** si vous fixez le paramètre `smooth_idf` à `False`, la formule n’est pas encore tout à fait celle que vous connaissez, puisqu’elle ajoute 1 au calcul du logarithme :

$$
\text{IDF}(d, N) = \ln{\frac{N}{d}} + 1
$$

### Calcul du TF-IDF

Il ne reste plus qu’à multiplier les deux mesures pour obtenir le TF-IDF de *devoirs* :

In [17]:
tfidf = list(map(lambda tf: tf * idf, tfs))

print(tfidf)

[2.0, 1.0, 0.0]


### Normalisation du TF-IDF

Là encore, le constructeur expose un paramètre pour appliquer ou non une couche de normalisation. Si `norm` est à `l2`, sa valeur par défaut, alors la somme des carrés des éléments du vecteur vaudra 1.

## Vérification

Considérez un corpus encore plus restreint afin de faciliter la vérification de ces principes :

In [18]:
corpus = [
    "little cats are nicer than old cats",
    "cats don't stay little for long"
]

Obtenez la matrice TF-IDF de ce corpus en limitant à un vocabulaire volontairement réduit :

In [19]:
vocabulary = ["little", "cats", "old", "long"]
vectorizer = TfidfVectorizer(vocabulary=vocabulary)
X = vectorizer.fit_transform(corpus)

Affichez le résultat :

In [20]:
print(X)

  (0, 2)	0.5321543559503558
  (0, 1)	0.7572644142929534
  (0, 0)	0.3786322071464767
  (1, 3)	0.7049094889309326
  (1, 1)	0.5015489070943787
  (1, 0)	0.5015489070943787


À présent, construisez à la main une matrice des occurrences des termes du vocabulaire pour représenter le TF de chaque terme dans le vecteur :

In [21]:
# voc: 'little', 'cats', 'old', 'long'
A = [1, 2, 1, 0]
B = [1, 1, 0, 1]

Appliquez la formule IDF pour chaque mot :

In [22]:
little = ln((1 + 2) / (1 + 2)) + 1
cats   = ln((1 + 2) / (1 + 2)) + 1
old    = ln((1 + 2) / (1 + 1)) + 1
long   = ln((1 + 2) / (1 + 1)) + 1

Trouvez maintenant leur mesure TF-IDF :

In [23]:
tfidf_A = [1 * little, 2 * cats, 1 * old, 0 * long]
tfidf_B = [1 * little, 1 * cats, 0 * old, 1 * long]

Vous obtenez ainsi la matrice creuse :

In [24]:
import numpy as np

np.array([
    tfidf_A,
    tfidf_B
])

array([[1.        , 2.        , 1.40546511, 0.        ],
       [1.        , 1.        , 0.        , 1.40546511]])

Chaque vecteur est ensuite normalisé en divisant le TF-IDF par la racine carrée de la somme des carrés du vecteur :

In [25]:
tfidf_A_little = 1 / ( (1 ** 2) + (2 ** 2) + (1.40546511 ** 2) ) ** 0.5
tfidf_A_cats   = 2 / ( (1 ** 2) + (2 ** 2) + (1.40546511 ** 2) ) ** 0.5
tfidf_A_old    = 1.40546511 / ( (1 ** 2) + (2 ** 2) + (1.40546511 ** 2) ) ** 0.5
tfidf_A_long   = 0 / ( (1 ** 2) + (2 ** 2) + (1.40546511 ** 2) ) ** 0.5

tfidf_B_little = 1 / ( (1 ** 2) + (1 ** 2) + (1.40546511 ** 2) ) ** 0.5
tfidf_B_cats   = 1 / ( (1 ** 2) + (1 ** 2) + (1.40546511 ** 2) ) ** 0.5
tfidf_B_old    = 0 / ( (1 ** 2) + (1 ** 2) + (1.40546511 ** 2) ) ** 0.5
tfidf_B_long   = 1.40546511 / ( (1 ** 2) + (1 ** 2) + (1.40546511 ** 2) ) ** 0.5

Si vous regardez par exemple le TF-IDF du terme *long* calculé par *Scikit-Learn* dans le deuxième document, vous trouverez 0.704909 :

In [26]:
df = pd.DataFrame(
    data=X.toarray(),
    columns=vectorizer.get_feature_names_out()
)

df

,little,cats,old,long
0,0.378632,0.757264,0.532154,0.000000
1,0.501549,0.501549,0.000000,0.704909


Un résultat identique à ce que vous venez de calculer à la main :

In [27]:
tfidf_B_long

0.7049094894083006